In [ ]:
%pip install bastionai

In [ ]:
%pip install torch==1.12.0 torchvision ipykernel ipywidgets

In [1]:
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.datasets import CIFAR100
from torchvision.transforms import ToTensor, Normalize, Compose

transform = Compose([
    ToTensor(),
    Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    lambda x: [x],
])

train_dataset = CIFAR100("data", train=True, transform=transform, download=True)
test_dataset = CIFAR100("data", train=False, transform=transform, download=True)

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
from bastionai.client import Connection

with Connection("localhost", 50051) as client:
    remote_dataloader = client.RemoteDataset(train_dataset, test_dataset, name="CIFAR100")

In [3]:
with Connection("localhost", 50051) as client:
    remote_learner = client.RemoteLearner(
        model,
        remote_dataloader,
        max_batch_size=4,
        loss="cross_entropy",
        model_name="ResNet50",
    )

    remote_learner.fit(nb_epochs=100, eps=6.0)
    # remote_learner.test(metric="accuracy")
    
    # trained_model = remote_learner.get_model()

Epoch 1/100 - train:   1%|                    | 67/12500 [00:58<3:00:11,  1.15batch/s, cross_entropy=6.8129 (+/- 0.0000)] 

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Connection reset by peer"
	debug_error_string = "{"created":"@1662545179.524951663","description":"Error received from peer ipv4:127.0.0.1:50051","file":"src/core/lib/surface/call.cc","file_line":966,"grpc_message":"Connection reset by peer","grpc_status":14}"
>